In [ ]:
# general packages
import numpy as np
# general packages
import numpy as np
from numpy import asarray
import napari
import time
import os
from datetime import date

# package for 3d visualization
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = [5, 5]

# package for io 
from aicsimageio import AICSImage
from aicsimageio.readers.czi_reader import CziReader

# function for core algorithm
from skimage.io import imsave
import cv2 as cv2
import scipy.ndimage
from patchify import patchify

In [ ]:
# load data from .czi format

# User inputs the file path
filePath = "//research.files.med.harvard.edu/sysbio/MEGASON LAB/People/AntoineRuzette/Data/Zebrafish neural tube/20220721_memNG_h2bScarlet_7hpf_40X-01-Airyscan Processing-01.czi"

# Load the image using the AICS importer
img = AICSImage(filePath, reader=CziReader)

# Image information
micronsPerPixel = [img.physical_pixel_sizes.Z, img.physical_pixel_sizes.X , img.physical_pixel_sizes.Y]
print(img.dims)
print(f'Scale of ZXY axis: [Z: {micronsPerPixel[0]}, X: {micronsPerPixel[1]}, Y: {micronsPerPixel[2]}]')

In [ ]:
# view first image - optional

# path to save
slice_path = "//research.files.med.harvard.edu/sysbio/MEGASON LAB/People/AntoineRuzette/Data/test_002.png"

# retrieve specific chunk
r = (img.get_image_dask_data('YX', T=1, Z=0, C=0)).compute()
g = (img.get_image_dask_data('YX', T=1, Z=0, C=1)).compute()

im_red = colorize(r, (1, 0, 0))
im_green = colorize(g, (0, 1, 0))
composite = np.clip(im_red + im_green, 0, 1)

# check exporting to png
imsave(slice_path, composite)


# The color we provide gives RGB values in order, between 0 and 1
plt.imshow(im_red)
plt.axis(False)
plt.title('Magenta')
plt.show()

plt.imshow(im_green)
plt.axis(False)
plt.title('Green')
plt.show()

plt.imshow(composite)
plt.title('Composite image')
plt.axis(False)
plt.show()


# channel axis expands on the dimension specified by its index in 'TCZYX'; if channel_axis = 0, creates a layer for each timestamp       
viewer = napari.view_image(im_red, contrast_limits=(9000, 23000), show = True)

# optional
viewer.scale_bar
viewer.axes.visible = True
viewer.axes.colored = False
viewer.scale_bar.visible = True
viewer.scale_bar.unit = "um"
viewer.scale_bar.ticks = False

In [ ]:
def colorize(im, color, clip_percentile=0.1):
    """
    Helper function to create an RGB image from a single-channel image using a 
    specific color.
    Reprinted from https://bioimagebook.github.io/chapters/1-concepts/4-colors/python.html, November 2022. 
    """
    # Check that we do just have a 2D image
    if im.ndim > 2 and im.shape[2] != 1:
        raise ValueError('This function expects a single-channel image!')
        
    # Rescale the image according to how we want to display it
    im_scaled = im.astype(np.float32) - np.percentile(im, clip_percentile)
    im_scaled = im_scaled / np.percentile(im_scaled, 100 - clip_percentile)
    im_scaled = np.clip(im_scaled, 0, 1)
    
    # Need to make sure we have a channels dimension for the multiplication to work
    im_scaled = np.atleast_3d(im_scaled)
    
    # Reshape the color (here, we assume channels last)
    color = np.asarray(color).reshape((1, 1, -1))
    return im_scaled * color

In [ ]:
# function selecting a certain slice at a certain timestamp

def slice(img, folder_path, counter, time: int, z_slice: int):
    """
    Helper function to slice a 4D stack into 2D images. 
    """
    
    if time > img.dims.T:
        print('Time is out of bound.')
    elif z_slice > img.dims.Z:
        print('Slice is out of bound.')
    else: 
        r = (img.get_image_dask_data('YX', T=time, Z=z_slice, C=0)).compute()
        g = (img.get_image_dask_data('YX', T=time, Z=z_slice, C=1)).compute()
        im_red = colorize(r, (1, 0, 0))
        im_green = colorize(g, (0, 1, 0))
        composite = np.clip(im_red + im_green, 0, 1)
        
        
        '''
        slice_name = f'img_{str(counter)}.png'
        slice_path = f'{folder_path}/{slice_name}'
        imsave(slice_path, composite)
        log = f'Slice (t{time}; z{z_slice}; counter: {counter}) successfully exported!'
        print(log)
        '''


        # TODO
        # make the image saving faster
        # retrieve metadata too

    return composite

In [ ]:
# function upsampling images then spliting them into patches
# return an array (list of list) of images

def resize(img, upsampling_size: tuple(), patch_size): 
    # assumes that the image array is square eg 1000x1000 px
    if upsampling_size[0] == upsampling_size[1]:
        upsampling_factor = int(upsampling_size[0]/img.shape[0])
        img_up = scipy.ndimage.zoom(img, upsampling_factor, order = 0)
    else: print('Images should be square, or upsampled')
    
    if patch_size[0] == patch_size[1]:
        patches = patchify.patchify(img_up, patch_size, step=patch_size[0])

    return patches

In [ ]:
# pull an user-specified chunk - sclicing for time and z dimensions

##################################################################################################################################
# loaded image
img = img

# time slicing
time1 = 0
time2 = 16
#img.dims.T
time_step = 1

# z slicing 
z1 = 0
z2 = img.dims.Z
z_step = 1

# upsampling size 
upsampling_size = (1024, 1024)

# patch size 
patch_size = (256, 256, 3)

# patch step 
patch_step = 250

# output folder
out = "//research.files.med.harvard.edu/sysbio/MEGASON LAB/People/AntoineRuzette/Data"
folder_name = 'NeuralTube_2D_raw03'
folder_path = f'{out}/{folder_name}_{str(date.today())}'
folder_img_path = f'{folder_path}/img'
###################################################################################################################################         

if ~os.path.isdir(folder_path):
    os.mkdir(folder_path)
    os.mkdir(folder_img_path)

idx = 0 
for time in range(time1, time2, time_step):
    #sub_folder_path = f'{folder_path}/T{str(time)}'
    #os.mkdir(sub_folder_path)
    
    for z in range(z1, z2, z_step): 
        slice_z, log = slice(img, folder_img_path, idx, time, z)
        slice_upsampled = scipy.ndimage.zoom(slice_z, upsampling_factor, order = 0)
        patches = patchify.patchify(slice_upsampled, patch_size, step=patch_step)

        for i in range(patches.shape[0]):
            for j in range(patches.shape[1]):
                patch = patches[i, j, 0]
                imsave(f'{folder_img_path}/patch_{idx}.png', patch)

                log_path = f'{folder_path}/log.txt'
                log = f'Slice (t{time}; z{z}; counter: {idx}) successfully exported!'
                print(log)

                # write in log file
                with open(log_path, 'a') as log_file:
                    log_file.write(log + '\n')

                idx+=1 